In [1]:
import tensorflow as tf

In [2]:
import pickle
import os
import tensorflow as tf

if os.path.exists('data.pkl'):
    with open('data.pkl', 'rb') as f:
        X, y = pickle.load(f)
else:
    X= tf.convert_to_tensor(X_reshaped, dtype=tf.float32)
    y=tf.convert_to_tensor(y_categorical, dtype=tf.float32)
    with open('data.pkl', 'wb') as f:
        pickle.dump((X, y), f)

In [3]:
split_index = int(0.8 * X.shape[0])  
X_train, X_test = tf.split(X, [split_index, X.shape[0] - split_index])
y_train, y_test = tf.split(y, [split_index, y.shape[0] - split_index])

# CNN

In [4]:
import random
from tensorflow.keras import layers, models

search_space = {
    'num_conv_layers': range(1, 4),
    'num_filters': [16, 32, 64],
    'filter_sizes': [(3, 3), (5, 5)],
    'num_dense_layers': range(1, 4),
    'dense_layer_units': [32, 64, 128]
}

population_size = 10
num_iterations = 100
c1 = 2.0
c2 = 2.0
w = 0.7
convergence = []
def evaluate_cnn_model(params, input_shape, num_classes, X_train, y_train, X_test, y_test):
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape, padding='same'))
    model.add(layers.MaxPooling2D((2, 2)))
    for _ in range(params['num_conv_layers']):
        model.add(layers.Conv2D(params['num_filters'], params['filter_sizes'], activation='relu', padding='same'))
        if model.output_shape[1] > 1 and model.output_shape[2] > 1:
            model.add(layers.MaxPooling2D((2, 2)))
        else:
            model.add(layers.MaxPooling2D((1, 1)))
    model.add(layers.Flatten())
    for _ in range(params['num_dense_layers']):
        model.add(layers.Dense(params['dense_layer_units'], activation='relu'))
    model.add(layers.Dense(num_classes, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(x=X_train, y=y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test), verbose=0)
    _, accuracy = model.evaluate(X_test, y_test, verbose=0)
    return accuracy

def initialize_population(population_size, search_space):
    population = []
    for _ in range(population_size):
        individual = {}
        for param, values in search_space.items():
            if param == 'filter_sizes':
                filter_size = random.choice(values)
                while 0 in filter_size:
                    filter_size = random.choice(values)
                individual[param] = filter_size
            else:
                individual[param] = random.choice(values)
        population.append(individual)
    return population


def pso(population_size, num_iterations, search_space, input_shape, num_classes, X_train, y_train, X_test, y_test):
    population = initialize_population(population_size, search_space)
    personal_best = population.copy()
    global_best = None
    global_best_fitness = -1

    for iteration in range(num_iterations):
        print(f"Iteration {iteration + 1}/{num_iterations}")
        for i, individual in enumerate(population):
            fitness = evaluate_cnn_model(individual, input_shape, num_classes, X_train, y_train, X_test, y_test)
            if fitness > evaluate_cnn_model(personal_best[i], input_shape, num_classes, X_train, y_train, X_test, y_test):
                personal_best[i] = individual.copy()
            if fitness > global_best_fitness:
                global_best = individual.copy()
                global_best_fitness = fitness

        convergence.append(global_best_fitness)

        velocity = [{param: [0, 0] if param == 'filter_sizes' else 0 for param in search_space.keys()} for _ in range(population_size)]

        for i, individual in enumerate(population):
            for param, value in search_space.items():
                if param == 'filter_sizes':
                    velocity[i][param] = (
                        w * velocity[i][param][0] + c1 * random.random() * (personal_best[i][param][0] - individual[param][0]) + c2 * random.random() * (global_best[param][0] - individual[param][0]),
                        w * velocity[i][param][1] + c1 * random.random() * (personal_best[i][param][1] - individual[param][1]) + c2 * random.random() * (global_best[param][1] - individual[param][1])
                    )
                    individual[param] = (
                        int(round(individual[param][0] + velocity[i][param][0])),
                        int(round(individual[param][1] + velocity[i][param][1]))
                    )
                else:
                    velocity[i][param] = w * velocity[i][param] + c1 * random.random() * (personal_best[i][param] - individual[param]) + c2 * random.random() * (global_best[param] - individual[param])
                    individual[param] = int(round(individual[param] + velocity[i][param]))
                    individual[param] = max(min(individual[param], max(value)), min(value))

    return global_best, global_best_fitness


input_shape = (22, 4, 1)
num_classes = 4
split_index = int(0.8 * X.shape[0])
X_train, X_test = tf.split(X, [split_index, X.shape[0] - split_index])
y_train, y_test = tf.split(y, [split_index, y.shape[0] - split_index])

best_individual, best_fitness = pso(population_size, num_iterations, search_space, input_shape, num_classes, X_train, y_train, X_test, y_test)
print("Best CNN Architecture found:")
print(best_individual)
print("Best Accuracy:", "%.3f" %best_fitness)


Iteration 1/100





Iteration 2/100
Iteration 3/100
Iteration 4/100
Iteration 5/100
Iteration 6/100


ValueError: The `kernel_size` argument must be a tuple of 2 integers. Received: (5, 0) including {0} that does not satisfy the requirement `> 0`.

# EEG-Net

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, BatchNormalization, ELU, Dropout, Dense, Flatten
from keras import regularizers
import tensorflow as tf

def build_classifier(input_shape, num_classes, num_filters_conv1, num_filters_conv2):
    classifier = Sequential()

    classifier.add(Conv2D(num_filters_conv1, 32, input_shape=input_shape, padding='same',
                          kernel_regularizer=regularizers.l2(0.01)))
    classifier.add(BatchNormalization())
    classifier.add(ELU())
    classifier.add(Dropout(0.5))
    
    for itr in range(1):
        classifier.add(Conv2D(num_filters_conv2, 8, padding='same',
                              kernel_regularizer=regularizers.l2(0.01)))
        classifier.add(BatchNormalization())
        classifier.add(ELU())
        classifier.add(Conv2D(num_filters_conv2, 1, padding='valid'))
        classifier.add(BatchNormalization())
        classifier.add(ELU())
        classifier.add(Dropout(0.5))
        classifier.add(Flatten())
        classifier.add(Dense(units=num_classes, kernel_initializer='uniform', activation='softmax'))
    
    return classifier

def evaluate_eeg_naet_model(num_filters_conv1, num_filters_conv2):
    classifier = build_classifier(input_shape=(22, 4, 1), num_classes=4,
                                  num_filters_conv1=int(num_filters_conv1),
                                  num_filters_conv2=int(num_filters_conv2))
    classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    history = classifier.fit(x=X_train, y=y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test), verbose=0)

    _, accuracy = classifier.evaluate(X_test, y_test)
    return -accuracy

def pso_optimization_eeg_naet(bounds, num_particles, num_iterations):
    particles = np.random.uniform(bounds[0], bounds[1], (num_particles, 2))
    personal_best_positions = particles.copy()
    personal_best_scores = np.array([evaluate_eeg_naet_model(p[0], p[1]) for p in particles])
    global_best_position = personal_best_positions[np.argmin(personal_best_scores)]
    global_best_score = np.min(personal_best_scores)

    convergence = []  # Initialize an array to track convergence

    for _ in range(num_iterations):
        for i, particle in enumerate(particles):
            velocity = np.random.uniform(-1, 1, 2)
            new_position = particle + velocity
            new_position = np.clip(new_position, bounds[0], bounds[1])
            particles[i] = new_position

            score = evaluate_eeg_naet_model(new_position[0], new_position[1])
            if score < personal_best_scores[i]:
                personal_best_positions[i] = new_position
                personal_best_scores[i] = score

                if score < global_best_score:
                    global_best_position = new_position
                    global_best_score = score

        convergence.append(global_best_score)

    return global_best_position, -global_best_score, convergence

bounds = [(16, 64), (16, 64)]

best_hyperparameters, best_accuracy, convergence = pso_optimization_eeg_naet(bounds, num_particles=10, num_iterations=100)

print("Best hyperparameters found:", best_hyperparameters)
print("Best accuracy found:", best_accuracy)


# MLP

In [ ]:
import numpy as np
from tensorflow.keras import layers, models
def create_mlp_model(input_shape, num_classes, num_neurons_dense1, num_neurons_dense2):
    model = models.Sequential()
    model.add(layers.Flatten(input_shape=input_shape))
    model.add(layers.Dense(num_neurons_dense1, activation='relu'))
    model.add(layers.Dense(num_neurons_dense2, activation='relu'))
    model.add(layers.Dense(num_classes, activation='softmax'))  
    return model

def evaluate_mlp_model(num_neurons_dense1, num_neurons_dense2):
    mlp_model = create_mlp_model(input_shape=(22, 4), num_classes=4,
                                 num_neurons_dense1=int(num_neurons_dense1),
                                 num_neurons_dense2=int(num_neurons_dense2))
    mlp_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    history = mlp_model.fit(x=X_train, y=y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test), verbose=0)

    _, accuracy = mlp_model.evaluate(X_test, y_test)
    return -accuracy

def pso_optimization_mlp(bounds, num_particles, num_iterations):
    particles = np.random.uniform(bounds[0], bounds[1], (num_particles, 2))
    personal_best_positions = particles.copy()
    personal_best_scores = np.array([evaluate_mlp_model(p[0], p[1]) for p in particles])
    global_best_position = personal_best_positions[np.argmin(personal_best_scores)]
    global_best_score = np.min(personal_best_scores)

    convergence = []

    for _ in range(num_iterations):
        for i, particle in enumerate(particles):
            velocity = np.random.uniform(-1, 1, 2)
            new_position = particle + velocity
            new_position = np.clip(new_position, bounds[0], bounds[1])
            particles[i] = new_position

            score = evaluate_mlp_model(new_position[0], new_position[1])
            if score < personal_best_scores[i]:
                personal_best_positions[i] = new_position
                personal_best_scores[i] = score

                if score < global_best_score:
                    global_best_position = new_position
                    global_best_score = score

        convergence.append(global_best_score)

    return global_best_position, -global_best_score, convergence

bounds = [(32, 256), (16, 128)]

best_hyperparameters, best_accuracy, convergence = pso_optimization_mlp(bounds, num_particles=10, num_iterations=100)

print("Best hyperparameters found:", best_hyperparameters)
print("Best accuracy found:", best_accuracy)


# RNN

In [ ]:
import random
import tensorflow as tf
from tensorflow.keras import layers, models

import seaborn as sns

search_space = {
    'num_units': [32, 64, 128],
    'num_layers': range(1, 4)
}

population_size = 10
num_iterations = 100
c1 = 2.0
c2 = 2.0
w = 0.7

def create_rnn_model(params, input_shape, num_classes):
    model = models.Sequential()
    model.add(layers.SimpleRNN(params['num_units'], activation='relu', input_shape=input_shape))

    for _ in range(params['num_layers']):
        model.add(layers.Dense(params['num_units'], activation='relu'))

    model.add(layers.Dense(num_classes, activation='softmax'))

    return model

def evaluate_rnn_model(params, input_shape, num_classes, X_train, y_train, X_test, y_test):
    model = create_rnn_model(params, input_shape, num_classes)

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(x=X_train, y=y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test), verbose=0)

    _, accuracy = model.evaluate(X_test, y_test, verbose=0)
    return accuracy

def pso(population_size, num_iterations, search_space, input_shape, num_classes, X_train, y_train, X_test, y_test):
    population = initialize_population(population_size, search_space)
    velocity = [{param: 0 for param in search_space.keys()} for _ in range(population_size)]
    personal_best = population.copy()
    global_best = None
    global_best_fitness = -1

    convergence = []

    for iteration in range(num_iterations):
        print(f"Iteration {iteration + 1}/{num_iterations}")
        for i, individual in enumerate(population):
            fitness = evaluate_rnn_model(individual, input_shape, num_classes, X_train, y_train, X_test, y_test)
            if fitness > evaluate_rnn_model(personal_best[i], input_shape, num_classes, X_train, y_train, X_test, y_test):
                personal_best[i] = individual.copy()
            if fitness > global_best_fitness:
                global_best = individual.copy()
                global_best_fitness = fitness

            for param in search_space.keys():
                velocity[i][param] = w * velocity[i][param] + c1 * random.random() * (personal_best[i][param] - individual[param]) + c2 * random.random() * (global_best[param] - individual[param])
                individual[param] = int(round(individual[param] + velocity[i][param]))
                individual[param] = max(min(individual[param], max(search_space[param])), min(search_space[param]))

        convergence.append(global_best_fitness)

    return global_best, global_best_fitness, convergence

def initialize_population(population_size, search_space):
    population = []
    for _ in range(population_size):
        individual = {param: random.choice(values) for param, values in search_space.items()}
        population.append(individual)
    return population

input_shape = (22, 4)
num_classes = 4
split_index = int(0.8 * X.shape[0])
X_train, X_test = tf.split(X, [split_index, X.shape[0] - split_index])
y_train, y_test = tf.split(y, [split_index, y.shape[0] - split_index])

best_individual, best_fitness = pso(population_size, num_iterations, search_space, input_shape, num_classes, X_train, y_train, X_test, y_test)
print("Best RNN Model Architecture found:")
print(best_individual)
print("Best Accuracy:", "%.3f" %best_fitness)


------------------------------------------